In [118]:
import numpy as np
import scipy.stats as stats

from numpy.fft import fft, fftshift, ifftshift
from scipy.interpolate import interp1d
from time import time

In [119]:
N_SIMS = 20000                           # FIXME: number of simulations
N_BLOCKS = 100
N_STEPS = 1                            # FIXME: number of steps (excluding t = 0)
T = 1                                    # FIXME: time horizon
S0 = 1                                   # initial stock price (not using here for simplicity)

dt = T / N_STEPS                         # time interval

# GBM parameters
K = 1.1
R = 0.05                                  # FIXME: risk-free interest rate
Q = 0.02
MU = R - Q                               # FIXME: drift coefficient (GBM)
SIGMA = 0.4                              # FIXME: diffusion coefficient

# Jump parameters (NCPP normal cumulative Possion process)
LAMBDA = 0.5                             # FIXME: jump rate
MU_J = -0.1                               # FIXME: jump mean
SIGMA_J = 0.15                           # FIXME: jump standard deviation

muABM = MU - 0.5 * SIGMA ** 2
muRN = muABM + LAMBDA * (np.exp(MU_J + 0.5 * SIGMA_J**2) - 1)

# Fourier parameters ---------------------------------------------------------------------------------------------
X_WIDTH = 6                                # TODO: Width of the interval
N_GRIDS = 2**8                             # TODO: Number of grid points
ALPHA = -1                                # TODO: Dampening factor for a call

In [120]:
def payoff(x, xi, ALPHA, K, L, U, C, theta):
    # Scale
    S = C * np.exp(x)

    # Payoff; see e.g. Green, Fusai, Abrahams 2010, Eq. (3.24)
    g = np.exp(ALPHA * x) * np.maximum(theta * (S - K), 0) * (S >= L) * (S <= U)

    # Analytical Fourier transform of the payoff
    l = np.log(L / C)  # lower log barrier
    k = np.log(K / C)  # log strike
    u = np.log(U / C)  # upper log barrier

    # Integration bounds
    if theta == 1:  # call
        a = max(l, k)
        b = u
    else:  # put
        a = min(k, u)
        b = l

    # Green, Fusai, Abrahams 2010 Eq. (3.26) with extension to put option
    xi2 = ALPHA + 1j * xi
    G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2) - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)

    # Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
    if ALPHA == 0:
        G[len(G) // 2] = C * (np.exp(b) - np.exp(a) - np.exp(k) * (b - a))
    elif ALPHA == -1:
        G[len(G) // 2] = C * (b - a + np.exp(k - b) - np.exp(k - a))

    return S, g, G

In [121]:
def charfunction(xi, parameters, flag=0):
    """
    Compute the characteristic function for different pricing problems.
    :param xi: Fourier space grid.
    :param parameters: Parameters for the distributions.
    :param flag: Flag for backward (0) or forward (1) characteristic function.
    :return: Characteristic function.
    """
    meancorrection = (parameters['rf'] - parameters['q']) * parameters['dt'] - \
                     np.log(charfunction0(-1j, parameters))
    F = np.exp(1j * meancorrection * xi) * charfunction0(xi, parameters)
    if flag == 0:
        F = np.conj(F)
    return F

def charfunction0(xi, parameters):
    """
    Compute the characteristic function for various distributions.
    :param xi: Fourier space grid.
    :param parameters: Parameters for the distributions.
    :return: Characteristic function.
    """
    distr_type = parameters['distr']
    
    if distr_type == 1:  # Normal
        m = parameters['m']
        s = parameters['s']
        F = np.exp(1j * xi * m - 0.5 * (s * xi)**2)

    elif distr_type == 3:  # VG
        theta = parameters['theta']
        s = parameters['s']
        nu = parameters['nu']
        F = (1 - 1j * xi * theta * nu + 0.5 * nu * (s * xi)**2)**(-1 / nu)

    elif distr_type == 6:  # Kou
        s = parameters['s']
        lambd = parameters['lambda']
        pigr = parameters['pigr']
        eta1 = parameters['eta1']
        eta2 = parameters['eta2']
        F = np.exp(-0.5 * (s * xi)**2 + lambd * ((1 - pigr) * eta2 / (eta2 + 1j * xi) + pigr * eta1 / (eta1 - 1j * xi) - 1))

    elif distr_type == 7:  # Merton
        s = parameters['s']
        alpha = parameters['alpha']
        lambd = parameters['lambda']
        delta = parameters['delta']
        # F = np.exp(              -0.5 * (s * xi)**2 + lambd * (np.exp(1j * xi * alpha - 0.5 * (delta * xi)**2) - 1))
        # F = np.exp( 1j * xi * muRN - 0.5 * (s * xi)**2 + lambd * (np.exp(1j * xi * alpha - 0.5 * (delta * xi)**2) - 1))
        F = 1j * muRN * xi - 0.5 * (SIGMA * xi) ** 2 + LAMBDA*(np.exp(1j*MU_J*xi-0.5*(SIGMA_J*xi)**2) - 1) #characteristic exponent    
        F = np.exp(F) #characteristic function

    else:
        raise ValueError('Invalid distribution number')

    return F

In [122]:
def kernel(ngrid, xmin, xmax, parameters, alpha=0, disc=1, flag=0):
    """
    Set up grids, compute the characteristic function, and apply discounting.

    Parameters:
    ngrid (int): Number of grid points.
    xmin (float): Minimum value of the grid in real space.
    xmax (float): Maximum value of the grid in real space.
    parameters (dict): Dictionary of parameters for the characteristic function.
    alpha (float): Shift parameter, especially for Feng-Linetsky and convolution.
    disc (int): Discount factor in the density (0 for no, 1 for yes).
    flag (int): Type of characteristic function (0 for backward, 1 for forward).

    Returns:
    tuple: Grid in real space (x), density (h), grid in Fourier space (xi), Fourier transform of density (H).
    """

# N = N_GRIDS // 2
# b = X_WIDTH / 2  # upper bound of the support in real space
# dx = X_WIDTH / N_GRIDS  # grid step in real space
# x = dx * np.arange(-N, N)  # grid in real space
# dxi = np.pi / b  # Nyquist relation: grid step in Fourier space
# xi = dxi * np.arange(-N, N)  # grid in Fourier space

# param = parameters(distr = 7, T = T, dt = T, rf = R, q = Q)
# [x, fc, xi, psi_call] = kernel(N_GRIDS, -b, b, param, ALPHA, 0, 1)
# [x, fp, xi, psi_put] = kernel(N_GRIDS, -b, b, param, -ALPHA, 0, 1)

    N = ngrid // 2
    dx = (xmax - xmin) / ngrid
    x = dx * np.arange(-N, N)
    dxi = 2 * np.pi / (xmax - xmin)
    xi = dxi * np.arange(-N, N)

    # Assuming charfunction is defined elsewhere
    H = charfunction(xi + 1j * alpha, parameters, flag)  # characteristic function

    if disc == 1:
        H *= np.exp(-parameters.rf * parameters.dt)  # apply discount

    h = np.real(fftshift(fft(ifftshift(H)))) / (xmax - xmin)  # discounted kernel

    return x, h, xi, H

In [123]:
def parameters(distr, T, dt, rf, q=None):
    param = {'distr': distr, 'T': T, 'dt': dt, 'rf': rf, 'q': 0 if q is None else q}

    if distr == 1:  # Normal
        m = 0  # mean
        s = 0.4  # standard deviation
        param.update({'m': m, 's': s * (dt ** 0.5), 'lambdam': 0, 'lambdap': 0})

    elif distr == 3:  # VG
        C, G, M = 4, 12, 18
        nu = 1 / C
        theta = (1 / M - 1 / G) * C
        s = ((2 * C) / (M * G)) ** 0.5
        param.update({'nu': nu / dt, 'theta': theta * dt, 's': s * (dt ** 0.5), 'lambdam': -M, 'lambdap': G})

    elif distr == 6:  # Kou double exponential
        s, lambda_, pigr, eta1, eta2 = 0.1, 3, 0.3, 40, 12
        param.update({'s': s * (dt ** 0.5), 'lambda': lambda_ * dt, 'pigr': pigr, 'eta1': eta1, 'eta2': eta2, 'lambdam': -eta1, 'lambdap': eta2, 'FLc': (s ** 2) / 2, 'FLnu': 2})

    elif distr == 7:  # Merton jump-diffusion
        s, alpha, lambda_, delta = SIGMA, MU_J, LAMBDA, SIGMA_J
        param.update({'s': s * (dt ** 0.5), 'alpha': alpha, 'lambda': lambda_ * dt, 'delta': delta, 'lambdam': 0, 'lambdap': 0, 'FLc': (s ** 2) / 2, 'FLnu': 2})

    else:
        raise Exception('Invalid distribution type')

    return param


In [124]:
# # FOURIER SOLUTION ===============================================================================================
temp = time()

# Grids in real and Fourier space
N = N_GRIDS // 2
b = X_WIDTH / 2  # upper bound of the support in real space
dx = X_WIDTH / N_GRIDS  # grid step in real space
x = dx * np.arange(-N, N)  # grid in real space
dxi = np.pi / b  # Nyquist relation: grid step in Fourier space
xi = dxi * np.arange(-N, N)  # grid in Fourier space

param = parameters(distr = 7, T = T, dt = T, rf = R, q = Q)
[x, fc, xi, psi_call] = kernel(N_GRIDS, -b, b, param, ALPHA, 0, 1)
[x, fp, xi, psi_put] = kernel(N_GRIDS, -b, b, param, -ALPHA, 0, 1)

# Fourier transform of the payoff
# b = X_WIDTH / 2  # upper bound of the support in real space
U = S0 * np.exp(b)
L = S0 * np.exp(-b)
_, gc, Gc = payoff(x, xi, ALPHA, K, L, U, S0, 1)   # call
S, gp, Gp = payoff(x, xi, -ALPHA, K, L, U, S0, 0)  # put

# Discounted expected payoff computed with the Plancherel theorem
c = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gc * np.conj(psi_call))))) / X_WIDTH  # call
# call_fourier = interp1d(S, c, kind='cubic')(S0)
call_fourier = np.interp(S0, S, c, left=np.nan, right=np.nan)

p = np.exp(-R * T) * np.real(fftshift(fft(ifftshift(Gp * np.conj(psi_put))))) / X_WIDTH  # put
# put_fourier = interp1d(S, p, kind='cubic')(S0)
put_fourier = np.interp(S0, S, p, left=np.nan, right=np.nan)

# interp1d(S, p, kind='cubic')(S0) means that we interpolate the function p(S) at the point S0, S is the grid of the function p(S), S0 is in S

# # ================================================================================================================
t_fourier = time() - temp

/var/folders/p6/smj7c6c1417csf_h79x5rk4h0000gn/T/ipykernel_38139/4066493564.py:23: RuntimeWarning: invalid value encountered in divide
  G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2) - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)


In [125]:
# MONTE CARLO SOLUTION ===========================================================================================
temp = time()


call_mc_blocks = np.zeros(N_BLOCKS)
put_mc_blocks = np.zeros(N_BLOCKS)



# Monte Carlo simulation via MJD
for i in range(N_BLOCKS):
    # Compute NCPP
    N  = np.random.poisson(LAMBDA * dt, size = (N_STEPS, N_SIMS))
    J  = MU_J * N + SIGMA_J * np.sqrt(N) * np.random.randn(N_STEPS, N_SIMS)

    # # Compute ABM increments and add NCPP effects
    S = muRN * dt + SIGMA * np.sqrt(dt) * np.random.normal(size = (N_STEPS, N_SIMS))
    S += J

    # Combine to ABM
    S = np.vstack([
        np.zeros(N_SIMS),
        S.cumsum(axis = 0)
    ])
    # S = S0 * np.exp(X)
    S = S0 * np.exp(S[-1])

    call_mc_blocks[i] = np.exp(-R * T) * np.maximum(S - K, 0).mean()
    put_mc_blocks[i] = np.exp(-R * T) * np.maximum(K - S, 0).mean()
    
# Obtaining the results
call_mc = call_mc_blocks.mean()
put_mc = put_mc_blocks.mean()

# Standard error
call_mc_se = np.sqrt(call_mc_blocks.var() / N_BLOCKS)
put_mc_se = np.sqrt(put_mc_blocks.var() / N_BLOCKS)

# ================================================================================================================
t_mc = time() - temp

In [131]:
def fourier(ngrid, xwidth, alpha, muRN, sigma, T, S0, K, r):
    start_time = time()

    # Fourier transform method
    xwidth = 6  # width of the support in real space
    ngrid = 2 ** 8  # number of grid points
    alpha = -1  # damping factor for a call

    # Grids in real and Fourier space
    N = ngrid // 2
    b = xwidth / 2  # upper bound of the support in real space
    dx = xwidth / ngrid  # grid step in real space
    x = dx * np.arange(-N, N)  # grid in real space
    dxi = np.pi / b  # Nyquist relation: grid step in Fourier space
    xi = dxi * np.arange(-N, N)  # grid in Fourier space

    # Characteristic function at time T
    # to change it i should change psi,Psic,psi,Psip:
    # call
    xia = xi + 1j * alpha  
    psi = 1j * muRN * xia - 0.5 * (sigma * xia) ** 2 + LAMBDA*(np.exp(1j*MU_J*xia-0.5*(SIGMA_J*xia)**2)-1) #characteristic exponent
    Psic = np.exp(psi * T)  # characteristic function
    # put
    xia = xi - 1j * alpha 
    psi = 1j * muRN * xia - 0.5 * (sigma * xia) ** 2 + LAMBDA*(np.exp(1j*MU_J*xia-0.5*(SIGMA_J*xia)**2)-1) #characteristic exponent    
    Psip = np.exp(psi * T)  # characteristic function

    #these function provide the characteristic function of 8 Levy processes:
    # param = parameters(1,T,T,r,q); # set the parameters editing parameters.m
    # [x,fc,xi,Psic] = kernel(ngrid,-b,b,param,alpha,0,1); # call
    # [x,fp,xi,Psip] = kernel(ngrid,-b,b,param,-alpha,0,1); # put

    # Fourier transform of the payoff
    U = S0 * np.exp(b)
    L = S0 * np.exp(-b)
    _, gc, Gc = payoff(x, xi, alpha, K, L, U, S0, 1)  # call
    S, gp, Gp = payoff(x, xi, -alpha, K, L, U, S0, -1)  # put

    #Extra Figures:
    # Call option
    #figures_ft(S, x, xi, Psic, gc, Gc)
    # Put option
    #figures_ft(S, x, xi, Psip, gp, Gp)

    # Discounted expected payoff computed with the Plancherel theorem
    #final integral in pricing:
    c = np.exp(-r * T) * np.real(np.fft.fftshift(np.fft.fft(np.fft.ifftshift(Gc * np.conj(Psic))))) / xwidth  # call
    VcF = np.interp(S0, S, c, left=np.nan, right=np.nan)
    p = np.exp(-r * T) * np.real(np.fft.fftshift(np.fft.fft(np.fft.ifftshift(Gp * np.conj(Psip))))) / xwidth  # put
    VpF = np.interp(S0, S, p, left=np.nan, right=np.nan)
    cputime_F = ((time() - start_time))  
    print('{:20s}{:<14.10f}{:<14.10f}{:<14.10f}'.format('Fourier', VcF, VpF, cputime_F))
    
    return VcF, VpF, cputime_F

fourier(ngrid = N_GRIDS, xwidth = X_WIDTH, alpha = ALPHA, muRN = muRN, sigma = SIGMA, T = T, S0 = S0, K = K, r = R)

Fourier             0.0976350405  0.2435959747  0.0006649494  


/var/folders/p6/smj7c6c1417csf_h79x5rk4h0000gn/T/ipykernel_38139/4066493564.py:23: RuntimeWarning: invalid value encountered in divide
  G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2) - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)


(0.09763504054219659, 0.2435959747350855, 0.0006649494171142578)

In [126]:
# PRINT RESULTS ===================================================================================================
print(f"{'':18s}{'Call':15s}{'Put':15s}{'CPU Time/s':15s}{'Operations':15s}")
print(f"{'Fourier':15s}{call_fourier:15.10f}{put_fourier:15.10f}{t_fourier:15.10f}{N_GRIDS:13d}")
print(f"{'Monte Carlo':15s}{call_mc:15.10f}{put_mc:15.10f}{t_mc:15.10f}{N_BLOCKS * N_SIMS:13d}")
print(f"{'MC S.E.':15s}{call_mc_se:15.10f}{put_mc_se:15.10f}")

                  Call           Put            CPU Time/s     Operations     
Fourier           0.1361678125   0.2023215056   0.0010840893          256
Monte Carlo       0.0974091336   0.2436426728   0.1552429199      2000000
MC S.E.           0.0001923994   0.0001389116


In [127]:
# call_mc - put_mc - S0 * np.exp(-Q * T) + K * np.exp(-R * T)

# 0.097635 - 0.243595 - S0 * np.exp(-Q * T) + K * np.exp(-R * T)


In [128]:
np.random.rand(10) < 0.5

array([False, False,  True, False, False, False,  True, False,  True,
       False])